In [ ]:
import numpy as np
import equinox as eqx
import jax
import jax.numpy as jnp
import matplotlib.pyplot as plt
import importlib
import numpy as np
from jax import grad, jit, vmap
from jax.experimental.ode import odeint
from IPython.display import HTML

from typing import TYPE_CHECKING, Callable, Union, Optional

from VariablesClass import VariablesClass
from StructureClass import  StructureClass
from StateClass import StateClass
from EquilibriumClass import EquilibriumClass

import plot_funcs, colors, dynamics, helpers_builders, learning_funcs, file_funcs

In [ ]:
colors_lst, red, custom_cmap = colors.color_scheme()
plt.rcParams['axes.prop_cycle'] = plt.cycler('color', colors_lst)

## Params

In [ ]:
# --- Structure and init params ---

H, S = 1, 1  # # Hinges, # Shims per hinge

Nin = 3  # tip position in (x, y) and its angle
Nout = 3  # Fx, Fy, torque, all on tip

buckle  = jnp.ones((H, S), dtype=jnp.int32)  # initial cubkle state, per shim
rows = jnp.array([0])
cols = jnp.array([0])     # for example, first shim of each row
buckle = buckle.at[rows, cols].set(-1)

In [ ]:
# --- training params --- 

T_training = 58  # total training set time (not time to reach equilibrium during every step)

alpha = 0.005  # learning rate

# desired_buckle_type = 'random'
# desired_buckle_rand_key = 169
desired_buckle_type = 'opposite'

# control also tip angle - True, else False
# control_tip_angle = True
control_tip_angle = False

# tip values to buckle shims - 'BEASTAL' for the BEASTAL scheme, else 'one_to_one'
update_scheme = 'one_to_one'

# dataset_sampling = 'uniform'  # random uniform vals for x, y, angle
dataset_sampling = 'almost flat'  # flat piece, single measurement

In [ ]:
# --- k params ---

# k_type = 'Experimental'
k_type = 'Numerical'

if k_type == 'Numerical':
    k_soft_uniform = 1.0
    k_stiff_uniform = 1.5
    thetas_ss_uniform = 1/2
    thresh_uniform = 1

In [ ]:
# --- Equilibrium params ---

k_stretch_ratio = 1.1e2
# for equilibrium state calculation
n_steps = int(1e3)
T_eq= 4e2  # time for equilibrium claculation, [s]
print('dt=', float(T_eq/n_steps))
damping = 0.3  # something*sqrt(k*m)
mass = 1e-2  # [kg]

In [ ]:
# --- automatic parameters / variables ---
if k_type == 'Numerical':
    k_soft  = jnp.ones((H, S), dtype=jnp.float32) * k_soft_uniform  # (Hinges, Shims) stiff. in soft direction, per shim
    k_stiff = jnp.ones((H, S), dtype=jnp.float32) * k_stiff_uniform  # (Hinges, Shims) stiff. in stiff direction, per shim
    thetas_ss = jnp.full((H, S), thetas_ss_uniform, dtype=jnp.float32)  # (Hinges, Shims) rest angles per shim
    thresh = jnp.full((H, S), thresh_uniform, dtype=jnp.float32)  # (Hinges, Shims) rest angles per shim
    file_name = None
else:
    k_soft  = None
    k_stiff = None
    thetas_ss = jnp.full((H, S), 1.03312, dtype=jnp.float32)  # (Hinges, Shims) rest angles per shim
    thresh = jnp.full((H, S), 1.96257, dtype=jnp.float32)  # (Hinges, Shims) rest angles per shim
    file_name = 'Roee_offset3mm_dl75.txt'

if desired_buckle_type == 'random':
    key = jax.random.PRNGKey(desired_buckle_rand_key)   # seed
    desired_buckle = jax.random.randint(key, (H, S), minval=-1, maxval=2)  # note: maxval is exclusive
    desired_buckle = desired_buckle.at[desired_buckle==0].set(-1)
elif desired_buckle_type == 'opposite':
    desired_buckle = -buckle

In [ ]:
import StructureClass
importlib.reload(StructureClass)
from StructureClass import StructureClass

# --- build geometry (all topology stays in StructureClass) ---
Strctr = StructureClass(hinges=H, shims=S, L=1)
if update_scheme == 'BEASTAL':
    Strctr._build_learning_parameters(Nin, Nout)

In [ ]:
import VariablesClass
importlib.reload(VariablesClass)
from VariablesClass import VariablesClass

# --- Initiate variables ---
Variabs = VariablesClass(Strctr,
                         k_type = k_type,
                         k_soft=k_soft,
                         k_stiff=k_stiff,
                         thetas_ss=thetas_ss,           # rest/target angles
                         thresh=thresh,                 # threshold to buckle shims
                         stretch_scale=k_stretch_ratio,            # k_stretch = 50 * max(k_stiff)
                         file_name = file_name
                         )

In [ ]:
import SupervisorClass
importlib.reload(SupervisorClass)
from SupervisorClass import SupervisorClass

Sprvsr = SupervisorClass(Strctr, alpha, T_training, desired_buckle, control_tip_angle=control_tip_angle,
                         update_scheme=update_scheme)
print('desired_tau_in_t after init ', Sprvsr.desired_tau_in_t)
Sprvsr.create_dataset(Strctr, sampling=dataset_sampling)
print('tip positions=', Sprvsr.tip_pos_in_t)
print('desired buckle=', Sprvsr.desired_buckle_arr)

## Desired values

In [ ]:
import StateClass
importlib.reload(StateClass)
from StateClass import StateClass

# --- state (straight chain, unit spacing => rest lengths = 1) ---
State_meas = StateClass(Variabs, Strctr, Sprvsr, buckle_arr = buckle)  # measurement state, buckle defaults to +1
State_des = StateClass(Variabs, Strctr, Sprvsr, buckle_arr = buckle)  # desired states a.f.o training set

## One shot - choose tip position

In [ ]:
# --- state (straight chain, unit spacing => rest lengths = 1) ---
State = StateClass(Variabs, Strctr, Sprvsr, buckle_arr = buckle)  # buckle defaults to +1
print('State_notip buckle', State.buckle_arr)

In [ ]:
if k_type == 'Experimental':
    thetas, torques, ks, torque_of_theta, k_of_theta = file_funcs.build_torque_stiffness_from_file(file_name, savgol_window=9)
    thetas = thetas[::4]
    torques = torques[::4]
    ks = ks[::4]
elif k_type == 'Numerical':
    thetas = np.concatenate((np.arange(-np.pi/2, np.pi/2, np.pi/(T_training/2)), -np.arange(-np.pi/2, np.pi/2, np.pi/(T_training/2))))
    torques = np.zeros(np.size(thetas))
    B = -1
    for i, theta in enumerate(thetas):
        T = thetas[i, None]                 # (H,1)
        TH = Variabs.thetas_ss[:, None]     # (H,1) 
        # spring constant is position dependent
        stiff_mask = ((B == 1) & (T < TH)) | ((B == -1) & (T > -TH))  # thetas are counter-clockwise    
        k_rot_state = jnp.where(stiff_mask, Variabs.k_stiff, Variabs.k_soft)  # (H,S)
        torques[i] = jnp.sum(k_rot_state * (T - B*TH))   
        if B == 1 and T < -Variabs.thresh:  # buckle up since thetas are CCwise
            B_nxt = -1
            print('buckled up, theta=', T)
        elif B == -1 and T > Variabs.thresh:  # buckle down, thetas are CCwise
            B_nxt = 1
            print('buckled down, theta=', T)
        else:
            B_nxt = B
        B = B_nxt
thetas_degs = np.rad2deg(thetas)

In [ ]:
np.arange(np.pi/2, -np.pi/2, np.pi/58)

In [ ]:
plt.plot(thetas, torques)
plt.show()
# plt.plot(thetas_degs, ks)
# plt.show()

In [ ]:
import EquilibriumClass
importlib.reload(EquilibriumClass)
from EquilibriumClass import EquilibriumClass

# a=0.15
# b=1/11
# tip_pos = Sprvsr.tip_pos_in_t[1]
# tip_angle = Sprvsr.tip_angle_in_t[1]

# # --- initialize, no tip movement yet
# if State.pos_arr is not None:  # calculate equilib from state of previous run, if happened
#     Eq = EquilibriumClass(Strctr, T_eq, n_steps, damping, mass, buckle_arr=buckle, pos_arr=State.pos_arr)
# else:  # use initial flat position
#     Eq = EquilibriumClass(Strctr, T_eq, n_steps, damping, mass, buckle_arr=buckle)
# final_pos, pos_in_t, vel_in_t, potential_force_in_t = Eq.calculate_state(Variabs, Strctr, tip_pos=tip_pos,
#                                                                          tip_angle=tip_angle)
# State._save_data(0, Strctr, final_pos, State.buckle_arr, potential_force_in_t, compute_thetas_if_missing=True,
#                  control_tip_angle = control_tip_angle)
# print('State_notip buckle', State.buckle_arr)
# plot_funcs.plot_arm(State.pos_arr , State.buckle_arr, np.rad2deg(State.theta_arr), Strctr.L,
#                     modality="measurement")

torque_sim = np.zeros(np.size(thetas))
for i, theta in enumerate(thetas):
    print('theta', np.rad2deg(theta))
    tip_angle = -np.pi + theta
    tip_pos = Strctr.L*np.array([1, 0]) + Strctr.L*np.array([np.cos(theta), np.sin(theta)])
    # print('tip_angle', tip_angle)
    print('tip_pos', tip_pos)
    
    # --- initialize, no tip movement yet
    if State.pos_arr is not None:  # calculate equilib from state of previous run, if happened
        Eq = EquilibriumClass(Strctr, T_eq, n_steps, damping, mass, buckle_arr=State.buckle_arr, pos_arr=State.pos_arr)
    else:  # use initial flat position
        Eq = EquilibriumClass(Strctr, T_eq, n_steps, damping, mass, buckle_arr=buckle_arr)
    final_pos, pos_in_t, vel_in_t, potential_force_in_t = Eq.calculate_state(Variabs, Strctr, tip_pos=tip_pos,
                                                                             tip_angle=None)
    State._save_data(i, Strctr, final_pos, State.buckle_arr, Forces=potential_force_in_t, compute_thetas_if_missing=True,
                     control_tip_angle=control_tip_angle)
    State.buckle(Variabs, Strctr, i, State_meas)
#     plot_funcs.plot_arm(State.pos_arr , State.buckle_arr, np.rad2deg(State.theta_arr), Strctr.L,
#                         modality="measurement") 
    torque_sim[i] = helpers_builders.torque(tip_angle, State.Fx, State.Fy)
    
# plt.plot(thetas, torques, thetas, torque_sim)

In [ ]:
State.buckle_arr

In [ ]:
fontsize = 16
plt.plot(thetas, torques, '-', thetas[:], -torque_sim[:], '--', linewidth=3.0)
plt.legend(['theoretical', 'numerical'], fontsize=fontsize)
plt.xlabel(r'$\theta\,\left[rad\right]$', fontsize=fontsize)
plt.ylabel(r'$\tau$', fontsize=fontsize)

In [ ]:
State.buckle_in_t

In [ ]:
plt.plot(potential_force_in_t[-18:,::2].T, '.')
plt.ylabel('forces x')
plt.show()
plt.plot(potential_force_in_t[-18:,1::2].T, '.')
plt.ylabel('forces y')
plt.show()

# np.shape(vel_in_t)
plt.plot(vel_in_t[-18:,:,0].T, '.')
plt.ylabel('velocities x')
plt.show()
plt.plot(vel_in_t[-18:,:,1].T, '.')
plt.ylabel('velocities y')
plt.show()

In [ ]:
importlib.reload(plot_funcs)

fig, anim = plot_funcs.animate_arm(pos_in_t, Strctr.L, interval_ms=100, save_path=f"arm_animation_t{0}.gif", fps=2,
                                   frames=60)

# Show inline animation (uses JavaScript/HTML5)
HTML(anim.to_jshtml())

In [ ]:
import EquilibriumClass
importlib.reload(EquilibriumClass)
from EquilibriumClass import EquilibriumClass

# --- initialize, no tip movement yet
if State.pos_arr is not None:  # calculate equilib from state of previous run, if happened
    Eq = EquilibriumClass(Strctr, T_eq, n_steps, damping, mass, buckle_arr=buckle, pos_arr=State.pos_arr)
else:  # use initial flat position
    Eq = EquilibriumClass(Strctr, T_eq, n_steps, damping, mass, buckle_arr=buckle)
final_pos, pos_in_t, vel_in_t, potential_force_in_t = Eq.calculate_state(Variabs, Strctr, tip_pos=tip_pos,
                                                                         tip_angle=tip_angle)
State._save_data(0, Strctr, final_pos, State.buckle_arr, potential_force_in_t, compute_thetas_if_missing=True,
                 control_tip_angle = control_tip_angle)
print('State_notip buckle', State.buckle_arr)
plot_funcs.plot_arm(State.pos_arr , State.buckle_arr, np.rad2deg(State.theta_arr), Strctr.L,
                    modality="measurement")

## Desired values contd

In [ ]:
for i, tip_pos in enumerate(Sprvsr.tip_pos_in_t):    
    
    if i == 0:
        Eq_des = EquilibriumClass(Strctr, T_eq, n_steps, damping, mass, buckle_arr=Sprvsr.desired_buckle_arr)
    else:
        Eq_des = EquilibriumClass(Strctr, T_eq, n_steps, damping, mass, buckle_arr=Sprvsr.desired_buckle_arr,
                                  pos_arr=final_pos)
    final_pos, pos_in_t, vel_in_t, potential_force_in_t = Eq_des.calculate_state(Variabs, Strctr, tip_pos=tip_pos,
                                                                                 tip_angle=Sprvsr.tip_angle_in_t[i])
    
    State_des._save_data(i, Strctr, pos_arr=final_pos, buckle_arr=State_des.buckle_arr, Forces=potential_force_in_t,
                         compute_thetas_if_missing=True, control_tip_angle = control_tip_angle)
    print('State tip forces ', [State_des.Fx, State_des.Fy])
    print('State tip torque ', State_des.tip_torque)
    Sprvsr.set_desired(final_pos, State_des.tip_pos[-2], State_des.tip_pos[-1], i, State_des.tip_torque)

State_meas._save_data(0, Strctr, State_meas.pos_arr, State_meas.buckle_arr, compute_thetas_if_missing=True,
                      control_tip_angle = control_tip_angle)
print('pre training configuration')
plot_funcs.plot_arm(State_meas.pos_arr , State_meas.buckle_arr, np.rad2deg(State_meas.theta_arr),Strctr.L,
                    modality='measurement')

In [ ]:
Sprvsr.desired_tau_in_t

## Training

In [ ]:
import StateClass
importlib.reload(StateClass)
from StateClass import StateClass
importlib.reload(helpers_builders)

# --- state (straight chain, unit spacing => rest lengths = 1) ---
State_meas = StateClass(Variabs, Strctr, Sprvsr, buckle_arr = buckle)  # buckle defaults to +1
State_update = StateClass(Variabs, Strctr, Sprvsr, buckle_arr = buckle)  # buckle defaults to +1

# --- initialize, no tip movement yet
Eq = EquilibriumClass(Strctr, T_eq, n_steps, damping, mass, buckle_arr=State_meas.buckle_arr, pos_arr=State_meas.pos_arr)

State_meas._save_data(0, Strctr, State_meas.pos_arr, State_meas.buckle_arr, compute_thetas_if_missing=True,
                      control_tip_angle = control_tip_angle)
print('pre training configuration')
plot_funcs.plot_arm(State_meas.pos_arr , State_meas.buckle_arr, np.rad2deg(State_meas.theta_arr), Strctr.L,
                    modality="measurement")

In [ ]:
import SupervisorClass
importlib.reload(SupervisorClass)
from SupervisorClass import SupervisorClass

In [ ]:
importlib.reload(plot_funcs)

for t in range(Sprvsr.T):
    print('t=', t)   
    
    ## MEASUREMENT
    
    # --- position tip ---
    State_meas.position_tip(Sprvsr, t)
    
    # --- equilibrium ---
    final_pos, pos_in_t, vel_in_t, potential_force_in_t = Eq.calculate_state(Variabs, Strctr, tip_pos=State_meas.tip_pos,
                                                                             tip_angle=State_meas.tip_angle)
#     edge_lengths = vmap(lambda e: Strctr._get_edge_length(final_pos, e))(jnp.arange(Strctr.edges))
#     print('edge lengths', helpers_builders.numpify(edge_lengths))

    # --- save sizes and plot ---
    State_meas._save_data(t, Strctr, final_pos, State_meas.buckle_arr, potential_force_in_t, compute_thetas_if_missing=True,
                          control_tip_angle = control_tip_angle)
    plot_funcs.plot_arm(final_pos, State_meas.buckle_arr, np.rad2deg(State_meas.theta_arr), Strctr.L,
                        modality = "measurement")
    print('Forces', potential_force_in_t[-1])
    print('Fx on tip, measurement', State_meas.Fx)
    print('Fy on tip, measurement', State_meas.Fy)
    plt.plot(potential_force_in_t[-1,:], '.')
    plt.show()
    fig, anim = plot_funcs.animate_arm(pos_in_t, Strctr.L, interval_ms=20, save_path=f"arm_animation_t{t}.gif", fps=4,
                                   frames=20)

    # Show inline animation (uses JavaScript/HTML5)
    HTML(anim.to_jshtml())
    print('torque on tip, measurement', State_meas.tip_torque)
    
    fig, anim = plot_funcs.animate_arm(pos_in_t, Strctr.L, interval_ms=20, save_path=f"arm_animation_t{t}.gif", fps=4,
                                       frames=20)
    
    # --- loss ---
    Sprvsr.calc_loss(t, State_meas.Fx, State_meas.Fy, tau=State_meas.tip_torque)
    print('desired Fx', Sprvsr.desired_Fx_in_t[t])
    print('desired Fy', Sprvsr.desired_Fy_in_t[t])
    print('loss', Sprvsr.loss)
    
    ## UPDATE
    
    if t == 0:
        Sprvsr.calc_update_tip(t, Strctr, Variabs, State_meas,
                               current_tip_angle = State_meas.tip_angle,
                               prev_tip_update_pos = np.array([Strctr.L*Strctr.edges, 0.1]),
                               prev_tip_update_angle = 0.0)
    else:
        Sprvsr.calc_update_tip(t, Strctr, Variabs, State_meas,
                               current_tip_angle = State_meas.tip_angle)
    print('update_tip', Sprvsr.tip_pos_update_in_t)
    print('update angle', Sprvsr.tip_angle_update_in_t)
    
    
    # --- position tip ---
    State_update.position_tip(Sprvsr, t, "update")
    
    # --- equilibrium ---
    final_pos, pos_in_t, vel_in_t, potential_force_in_t = Eq.calculate_state(Variabs, Strctr, tip_pos=State_update.tip_pos,
                                                                             tip_angle=State_update.tip_angle)

    # --- save sizes and plot ---
    State_update._save_data(t, Strctr, final_pos, State_update.buckle_arr, potential_force_in_t,
                            compute_thetas_if_missing=True, control_tip_angle = control_tip_angle)
    plot_funcs.plot_arm(final_pos, State_update.buckle_arr, np.rad2deg(State_update.theta_arr),
                        Strctr.L, modality = "update")
    print('pre buckle', State_update.buckle_arr)
    # print('energy', Eq.energy(Variabs, Strctr, final_pos)[-1])
    
    # --- shims buckle ---
    State_update.buckle(Variabs, Strctr, t, State_measured = State_meas)
    
    Eq = EquilibriumClass(Strctr, T_eq, n_steps, damping, mass, buckle_arr=helpers_builders.jaxify(State_update.buckle_arr),
                          pos_arr=helpers_builders.jaxify(State_update.pos_arr))
    print('post buckle', State_update.buckle_arr)
    # print('post buckle update', State_update.buckle_arr)
    # print('energy', Eq.energy(Variabs, Strctr, final_pos)[-1])
    plot_funcs.plot_arm(final_pos, State_update.buckle_arr, np.rad2deg(State_update.theta_arr),
                        Strctr.L, modality = "update")   

In [ ]:
np.array([Sprvsr.desired_tau_in_t[t]-State_meas.tip_torque])

In [ ]:
plt.plot(np.sum(np.abs(Sprvsr.loss_in_t**2), axis=1))
plt.xlabel('t')
plt.ylabel('Loss')

In [ ]:
plt.plot(State_update.buckle_in_t[0,0,:])
plt.plot(State_update.buckle_in_t[1,0,:])
plt.plot(State_update.buckle_in_t[2,0,:])
plt.plot(State_update.buckle_in_t[3,0,:])
plt.plot(State_update.buckle_in_t[4,0,:])
plt.xlabel('t')
plt.ylabel('buckle')
plt.legend(['hinge 1', 'hinge 2', 'hinge 3', 'hinge 4', 'hinge 5'])

In [ ]:
fig, anim = plot_funcs.animate_arm(pos_in_t, Strctr.L, interval_ms=20, save_path=f"arm_animation_t{t}.gif", fps=4,
                                   frames=20)

# Show inline animation (uses JavaScript/HTML5)
HTML(anim.to_jshtml())

## Show what is static and what is not

In [ ]:
# import jax
# import jax.numpy as jnp
# import equinox as eqx

# params, static = eqx.partition(Variabs, eqx.is_inexact_array)

# # 1) Quick look: print the leaves that are considered “params”
# print("Param leaves (count):", len(jax.tree_util.tree_leaves(params)))

# # 2) See shapes/dtypes of param leaves
# leaf_summary = jax.tree_util.tree_map(
#     lambda x: (getattr(x, "shape", None), getattr(x, "dtype", None)), params
# )
# print(leaf_summary)

# # 3) If you want a compact structure printout:
# print("PARAM STRUCT:", jax.tree_util.tree_structure(params))
# print("STATIC STRUCT:", jax.tree_util.tree_structure(static))
